# Métodos de optimización no lineal sin restricciones
----------------

## Autores
* Quesada, Francisco
* Serpe, Octavio
* Arca, Gonzalo

-----------------

## Librerías utilizadas
* SciPy
* NumPy
-----------------


In [82]:
import math
from scipy.optimize import minimize
import numpy as np
import time
from autograd.misc.optimizers import adam
import numdifftools as nd
import matplotlib.pyplot as plt

In [73]:
MIN_REAL = -10
MAX_REAL = 10

NUMBER_OF_PARAMETERS = 11

DATASET_INPUT = [(4.4793, -4.0765, -4.0765), (-4.1793, -
                                              4.9218, 1.7664), (-3.9429, -0.7689, 4.8830)]

DATASET_OUTPUT = [0, 1, 1]

In [74]:
# W 3 element vector
# w 2x3 matrix as 6 element vector
# w_0 2 element vector
# reactive_values 3 element vector
def f(W, w, w_0, reactive_values):
    inner_g_input = 0
    outer_g_input = 0

    for j in range(2):
        for k in range(3):
            inner_g_input += w[k + (j * 3)] * reactive_values[k] - w_0[j]
        outer_g_input += W[j] * g(inner_g_input)
    return g(outer_g_input - W[0])


# dataset_input 3 element vector
# dataset_output 3 element vector
def error(x, dataset_input, dataset_output):
    #dataset_input, dataset_output = dataset_tuple
    error = 0

    W = x[0:3]
    w = x[3:9]
    w_0 = x[9:11]

    for i in range(len(dataset_input)):
        error += math.pow(dataset_output[i] -
                          f(W, w, w_0, dataset_input[i]), 2)

    return error


def g(x):
    # If over the limit, return the limit value
    try:
        exp_value = math.exp(x)
    except OverflowError:
        return 1

    return exp_value / (1 + exp_value)

In [75]:
def print_output(x, time):
  print(f'Argumentos óptimos:\nW = \t{x[0:3]}\nw = \t{x[3:6]}\n\t{x[6:9]}\nw_0 = \t{x[9:11]}\n')
  print(f'error: {error(x, DATASET_INPUT, DATASET_OUTPUT)}')
  print("Tiempo de ejecucion:", time, "segundos")

## Método 1: Gradiente descendiente (método quasi-Newton BFGS)

In [94]:
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-bfgs.html

x0 = np.random.uniform(low=MIN_REAL, high=MAX_REAL, size=NUMBER_OF_PARAMETERS)
start_time = time.perf_counter()
optimizeResult = minimize(error, x0, method='BFGS', args=(DATASET_INPUT, DATASET_OUTPUT), tol=1e-10, options={"disp":True})
end_time = time.perf_counter()
print_output(optimizeResult.x, end_time - start_time)


Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 18
         Function evaluations: 348
         Gradient evaluations: 29
Argumentos óptimos:
W = 	[120.31511331  12.49538164  -0.44308898]
w = 	[-0.07014637  1.98101769  8.15875882]
	[ 8.86167311 -1.89650646  0.26949546]
w_0 = 	[-4.84091046 -7.38824611]

error: 2.8033628296407015e-11
Tiempo de ejecucion: 0.012781590994563885 segundos


## Método 2: Gradientes conjugados

In [77]:
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-cg.html

x0 = np.random.uniform(low=MIN_REAL, high=MAX_REAL, size=NUMBER_OF_PARAMETERS)
start_time = time.perf_counter()
optimizeResult = minimize(error, x0, method='CG', args=(DATASET_INPUT, DATASET_OUTPUT), tol=1e-10, options={"disp":True})
end_time = time.perf_counter()
print_output(optimizeResult.x, end_time - start_time)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 15
         Function evaluations: 2064
         Gradient evaluations: 172
Argumentos óptimos:
W = 	[23.57285064 12.8033453  -7.87426544]
w = 	[-22.02913189 -28.28594493  11.52666476]
	[-21.05058058 -13.83270459  -0.63832122]
w_0 = 	[-9.85813742 -6.82608443]

error: 1.515099540561219e-11
Tiempo de ejecucion: 0.0532665420032572 segundos


## Método 3: ADAM

### Implementación del método

In [95]:
x0 = np.random.uniform(low=MIN_REAL, high=MAX_REAL, size=NUMBER_OF_PARAMETERS)
gradient = nd.Gradient(error)
def grad(x,i):
    return gradient(x, DATASET_INPUT, DATASET_OUTPUT)

values = []
steps = 0
gradients = []

def save_values(x,i,g):
    values.append(x)
    gradients.append(g)
    steps = i

start_time = time.perf_counter()
optimizeResult = adam(grad,x0,step_size= 0.8,callback=save_values,num_iters=100)
end_time = time.perf_counter()
print_output(optimizeResult, end_time - start_time) 


Argumentos óptimos:
W = 	[ 9.93544931  7.56192883 -5.94339096]
w = 	[-8.49543877  2.38929764  3.89335269]
	[-3.92431511 -1.84599246  9.67275032]
w_0 = 	[-12.98182375   0.73115958]

error: 5.423162050837844e-07
Tiempo de ejecucion: 0.6321328709964291 segundos
